In [1]:
!pip install transformers pymilvus sentence-transformers huggingface-hub langchain_community langchain-text-splitters pypdf


  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 27.4 MB/s eta 0:00:00 0:00:01
Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl (2.7 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2


In [2]:
! pip install bs4

In [3]:
import glob
path = '/Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/'
files = glob.glob(path +'*.html')

1. Extracting text

In [4]:

from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

text_lines = []

for file in files:
    with open(file, 'r', encoding='ISO-8859-1') as f:
        print(f"Processing: {file}")
        raw_html = f.read()

        # Clean HTML
        soup = BeautifulSoup(raw_html, "html.parser")
        clean_text = soup.get_text(separator=" ", strip=True)
        clean_text = " ".join(clean_text.split())  # Optional: remove excessive whitespace

        # Split cleaned text
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=20000, chunk_overlap=50)
        chunks = text_splitter.split_text(clean_text)

        # Convert to Document objects with metadata
        for chunk in chunks:
            text_lines.append(Document(page_content=chunk, metadata={"source": file}))


Processing: /Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Résumé des caractéristiques du produit - KARDEGIC 160 mg, poudre pour solution buvable en sachet - Base de données publique des médicaments.html
Processing: /Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Fiche info - SERESTA 10 mg, comprimé - Base de données publique des médicaments.html
Processing: /Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Fiche info - KARDEGIC 160 mg, poudre pour solution buvable en sachet - Base de données publique des médicaments.html
Processing: /Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Fiche info - DOLIPRANE 500 mg, comprimé - Base de données publique des médicaments.html
Processing: /Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Résumé des caractéristiques du produit - DAFALGAN 500 mg, comprimé - Base de données publique des médicaments.html
Processing: /Users/naomi/Desktop/EM/Deep learning/deeple

In [5]:
path = '/Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/'
files = glob.glob(path +'*.html')

In [6]:
text_lines = []
for file in files:
    with open(file, 'r',encoding='ISO-8859-1') as f:
        
        print(file)
        text = f.read()
        soup = BeautifulSoup(text, "html.parser")
        clean_text = soup.get_text(separator=" ", strip=True)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=20000, chunk_overlap=50)
        chunks = text_splitter.split_text(clean_text)
        text_lines += [chunk for chunk in chunks]
        
        

/Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Résumé des caractéristiques du produit - KARDEGIC 160 mg, poudre pour solution buvable en sachet - Base de données publique des médicaments.html
/Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Fiche info - SERESTA 10 mg, comprimé - Base de données publique des médicaments.html
/Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Fiche info - KARDEGIC 160 mg, poudre pour solution buvable en sachet - Base de données publique des médicaments.html
/Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Fiche info - DOLIPRANE 500 mg, comprimé - Base de données publique des médicaments.html
/Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Résumé des caractéristiques du produit - DAFALGAN 500 mg, comprimé - Base de données publique des médicaments.html
/Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Notice patient - DAFALGAN 500 mg, comprimé - Base de

2. Embedding a sentence

In [7]:
from sentence_transformers import SentenceTransformer

In [8]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") 
s = text_lines[0]
e = embedding_model.encode([s])

3. Creating a Milvus data collection


In [9]:
from pymilvus import MilvusClient

In [10]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") 

def emb_text (text_lines):
    embeddings = embedding_model.encode(text_lines, batch_size=32, show_progress_bar=False)
    return embeddings

In [11]:
milvus_client = MilvusClient(uri="./my_milvus_db.db")
collection_name = "rag_collection"
milvus_client.create_collection(
 collection_name=collection_name,
 dimension= 384, # The size of the embedding
 metric_type="IP", # Inner product distance
 consistency_level="Strong", # Strong consistency level
)


Open /Users/naomi/Desktop/EM/Deep learning/deeplearningproject/my_milvus_db.db failed, the file has been opened by another program
Start local milvus failed
2025-04-06 23:01:24,344 [ERROR][_create_connection]: Failed to create new connection using: 574c0ea00e894b96b002a81693764780 (milvus_client.py:923)


ConnectionConfigException: <ConnectionConfigException: (code=1, message=Open local milvus failed)>

In [12]:
data = []
# In the following example, emb_text is a function that needs to be written, based on
# an embedding model
for i, line in enumerate(text_lines):
    data.append({"id": i, "vector": emb_text(line), "text": line})


In [13]:
insert_res = milvus_client.insert(collection_name=collection_name, data=data)

NameError: name 'milvus_client' is not defined

4. Retrieving data for a query

In [150]:
question = "comment prendre doliprane?"
search_res = milvus_client.search(
 collection_name=collection_name,
 data=[
 emb_text(question)
 ],
 limit=3, # Return top 3 results
 search_params={"metric_type": "IP", "params": {}}, # Inner product distance
 output_fields=["text"], # Return the text field
)

In [151]:
context = [result['entity']['text'] for result in search_res[0]]
print(context)

["<p class=AmmCorpsTexte style='margin-top:0cm;margin-right:0cm;margin-bottom: 0cm;margin-left:14.2pt;margin-bottom:.0001pt;text-indent:-14.2pt'>1. Qu'est-ce que DOLIPRANE 500 mg, comprimé et dans quels cas est-il utilisé ?</p>\n<p class=AmmCorpsTexte style='margin-top:0cm;margin-right:0cm;margin-bottom: 0cm;margin-left:14.2pt;margin-bottom:.0001pt;text-indent:-14.2pt'>2. Quelles sont les informations à connaître avant de prendre DOLIPRANE 500 mg, comprimé ?</p>\n<p class=AmmCorpsTexte style='margin-top:0cm;margin-right:0cm;margin-bottom: 0cm;margin-left:14.2pt;margin-bottom:.0001pt;text-indent:-14.2pt'>3. Comment prendre DOLIPRANE 500 mg, comprimé ?</p>\n<p class=AmmCorpsTexte style='margin-top:0cm;margin-right:0cm;margin-bottom: 0cm;margin-left:14.2pt;margin-bottom:.0001pt;text-indent:-14.2pt'>4. Quels sont les effets indésirables éventuels ?</p>", "ne serait pas mentionné dans cette notice. Voir rubrique 4. · Vous devez vous adresser à votre médecin si la douleur persiste plus de 5 

5. Create a prompt 

In [152]:
from huggingface_hub import notebook_login
notebook_login()

from transformers import AutoTokenizer, AutoModelForCausalLM

In [153]:
model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [154]:
PROMPT = """
Use the information enclosed in <context> tags to provide an answer to the
question enclosed in <question> tags.
<context>
{context}
</context>
<question>
{question}
</question>" \
"""

In [155]:
PROMPT = PROMPT.format(question=question, context=context)
print(PROMPT)


Use the information enclosed in <context> tags to provide an answer to the
question enclosed in <question> tags.
<context>
["<p class=AmmCorpsTexte style='margin-top:0cm;margin-right:0cm;margin-bottom: 0cm;margin-left:14.2pt;margin-bottom:.0001pt;text-indent:-14.2pt'>1. Qu'est-ce que DOLIPRANE 500 mg, comprimé et dans quels cas est-il utilisé ?</p>\n<p class=AmmCorpsTexte style='margin-top:0cm;margin-right:0cm;margin-bottom: 0cm;margin-left:14.2pt;margin-bottom:.0001pt;text-indent:-14.2pt'>2. Quelles sont les informations à connaître avant de prendre DOLIPRANE 500 mg, comprimé ?</p>\n<p class=AmmCorpsTexte style='margin-top:0cm;margin-right:0cm;margin-bottom: 0cm;margin-left:14.2pt;margin-bottom:.0001pt;text-indent:-14.2pt'>3. Comment prendre DOLIPRANE 500 mg, comprimé ?</p>\n<p class=AmmCorpsTexte style='margin-top:0cm;margin-right:0cm;margin-bottom: 0cm;margin-left:14.2pt;margin-bottom:.0001pt;text-indent:-14.2pt'>4. Quels sont les effets indésirables éventuels ?</p>", "ne serait pa

In [156]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "gpt2" 
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map=None).to(device)



In [157]:
inputs = tokenizer(PROMPT.format(question=question, context=context), return_tensors="pt").to(device)
output_ids = model.generate(**inputs, max_new_tokens=100)


response = tokenizer.decode(output_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 -
<context>
<p class=AmmCorpsTexte style='margin-top:0cm;margin-right:0cm;margin-bottom: 0cm;margin-left:14.2pt;margin-bottom:.0001pt;text-indent:-14.2pt'>1. Qu'est-ce que DOLIPRANE 500 mg, comprimé et dans quels cas est-il utilisé ?

